In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
!pip install Faker

In [ ]:
import re
import pandas as pd
import random
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from tqdm import tqdm
from faker import Faker
from collections import defaultdict

# Загрузка модели и токенизатора для русского языка
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Перемещение модели на GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Генерация текста
# Изменение параметров генерации:
#     temperature: Управляет случайностью выборов модели. Более высокие значения
#     (например, 0.7, 1.0) приводят к большей случайности.
#     top_k: Ограничивает выбор следующего слова среди top_k самых вероятных слов.
#     Более низкие значения приводят к более разнообразным результатам.
#     top_p (nucleus sampling): Выбирает слова из вероятностной массы top_p.
#     Обычно используется значение между 0.8 и 0.9.

def generate_text(prompt, max_length=200, num_return_sequences=1,
                  num_beams=5, temperature=0.9, top_k=50, top_p=0.9):
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)

    # Генерация текста с помощью модели
    outputs = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        early_stopping=True,
        num_beams=num_beams,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        do_sample=True,
    )

    # Декодирование и возврат сгенерированных текстов
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]


def only_letters(text):
    words = re.findall('[а-яё]+', text.lower())
    return ' '.join(words)


def generate_fake_words(num_texts=100, min_words=5, max_words=30):
    faker = Faker("ru_RU")
    fake_words = []
    for _ in range(num_texts):
        num_words = random.randint(min_words, max_words)
        fake_words.append(faker.words(num_words))

    return fake_words


prompt_templates = [
    "Покупатель и продавец ведут беседу о характеристиках товара и условиях покупки.",
    "Клиент и менеджер обсуждают особенности и преимущества услуги.",
    "Диалог покупателя и продавца по поводу стоимости и качества товара.",
    "Клиент интересуется у продавца деталями предоставляемой услуги.",
    "Разговор покупателя и продавца о возможных скидках на товар.",
    "Клиент и менеджер обсуждают гарантийные условия и сервисное обслуживание.",
    "Покупатель спрашивает продавца о наличии и сроках доставки товара.",
    "Клиент уточняет у продавца информацию о дополнительных услугах.",
    "Диалог о выгодных предложениях и акциях на товары между покупателем и продавцом.",
    "Покупатель интересуется у продавца условиями возврата и обмена товара.",
    "Обсуждение клиента и менеджера по поводу расценок на услугу.",
    "Покупатель и продавец обсуждают сроки выполнения услуги.",
    "Клиент спрашивает продавца о возможностях персонализации товара.",
    "Покупатель и продавец ведут переговоры о специальной скидке на крупную покупку.",
    "Клиент интересуется у менеджера отзывами других клиентов о товаре.",
    "Покупатель спрашивает продавца о возможных способах оплаты.",
    "Разговор покупателя и продавца о совместимых аксессуарах к товару.",
    "Клиент обсуждает с продавцом условия подписки на услугу.",
    "Покупатель уточняет у продавца информацию о безопасности и сертификации товара.",
    "Клиент и менеджер ведут беседу о преимуществах товара по сравнению с конкурентами.",
    "Клиент и менеджер обсуждают возможные варианты кастомизации товара или услуги.",
    "Покупатель интересуется у продавца актуальными новинками и тенденциями в отрасли.",
    "Диалог о преимуществах использования продукции данного бренда по сравнению с другими.",
    "Покупатель и продавец обсуждают индивидуальные потребности клиента и наиболее подходящие решения.",
    "Клиент уточняет у менеджера информацию о процессе установки или использования товара.",
    "Разговор о предложениях финансирования или кредитных программ для покупки товара.",
    "Покупатель интересуется у продавца возможностью тестирования товара перед покупкой.",
    "Диалог о вариантах доставки и установки товара в удобное для клиента время.",
    "Клиент и менеджер обсуждают планы и перспективы развития бизнеса после покупки товара или услуги.",
    "Покупатель уточняет у продавца информацию о производителе товара и его репутации на рынке.",
    "Обсуждение с продавцом возможности продления гарантии на товар или услугу.",
    "Покупатель и менеджер обсуждают программы лояльности и преимущества для постоянных клиентов.",
    "Клиент интересуется у продавца дополнительными материалами или обучающими ресурсами по использованию товара.",
    "Диалог о возможности заказа товара на специальные мероприятия или мероприятия.",
    "Разговор о влиянии товара или услуги на окружающую среду и социальную ответственность компании.",
    "Клиент уточняет у менеджера информацию о профессиональной поддержке и консультациях по использованию товара.",
    "Обсуждение возможностей технической поддержки и обслуживания товара после покупки.",
    "Покупатель и продавец обсуждают возможности интеграции товара или услуги в текущие процессы и системы клиента.",
    "Клиент интересуется у продавца возможностью приобретения товара в рассрочку или на условиях аренды.",
    "Диалог между клиентом и менеджером о выборе оптимальной конфигурации товара.",
    "Покупатель уточняет у продавца информацию о сроках гарантии на товар.",
    "Разговор о возможности предоставления дополнительных услуг в комплексе с товаром.",
    "Клиент интересуется у продавца условиями программы лояльности.",
    "Обсуждение вариантов доставки товара с учетом особенностей географического расположения клиента.",
    "Покупатель и менеджер обсуждают преимущества онлайн-покупки по сравнению с офлайн магазинами.",
    "Диалог о перспективах развития сотрудничества между клиентом и компанией.",
    "Клиент уточняет у продавца информацию о технических характеристиках товара.",
    "Разговор о возможности приобретения товара в кредит или на условиях рассрочки.",
    "Покупатель интересуется у продавца мнением о популярности и востребованности товара.",
    "Обсуждение планов клиента по дальнейшему использованию приобретенного товара.",
    "Клиент и менеджер обсуждают возможность проведения презентации товара перед покупкой.",
    "Разговор о способах ухода и обслуживания товара после его приобретения.",
    "Покупатель интересуется у продавца информацией о производителе и стране производства товара.",
    "Обсуждение технической поддержки и консультации по использованию товара.",
    "Клиент и менеджер ведут беседу о возможностях индивидуального заказа товара.",
    "Разговор о вариантах упаковки и доставки товара с учетом его хрупкости.",
    "Покупатель уточняет у продавца информацию о наличии товара на складе и возможности его заказа.",
    "Обсуждение особенностей и условий использования гарантийного талона на товар.",
    "Клиент интересуется у менеджера информацией о сроках проведения акций и распродаж.",
    "Диалог о возможности приобретения дополнительной комплектации или аксессуаров к товару.",
    "Покупатель и продавец обсуждают особенности оформления заказа на товар через интернет.",
    "Разговор о возможности получения консультации специалиста перед покупкой товара.",
    "Клиент уточняет у продавца информацию о сроках и условиях действия гарантии на товар.",
    "Обсуждение перспектив сотрудничества в рамках программы партнерства.",
    "Диалог о вариантах оплаты и условиях доставки товара за рубеж.",
    "Покупатель интересуется у продавца информацией о возможности возврата товара по истечении срока гарантии.",
    "Разговор о возможности приобретения дополнительной гарантии на товар.",
    "Клиент и менеджер обсуждают условия возврата товара в случае его некачественности или несоответствия.",
    "Обсуждение перспектив дальнейшего сотрудничества и предложения по расширению ассортимента товаров."
]

nums = ('один', 'два', 'три', 'четыре', 'пять', 'шесть', 'семь', 'восемь', 'девять',
        'десять', 'одиннадцать', 'двенадцать', 'тринадцать', 'четырнадцать', 'пятнадцать',
        'шестнадцать', 'семнадцать', 'восемнадцать', 'девятнадцать', 'двадцать',
        'двадцать один', 'двадцать два', 'двадцать три', 'двадцать четыре',
        'двадцать пять', 'двадцать шесть', 'двадцать семь', 'двадцать восемь',
        'двадцать девять', 'тридцать'
        )

percents = ('процент', 'процента', 'процентов', 'проценты',)

discount = ("скидка",  # Именительный падеж
            "скидки",  # Родительный падеж
            "скидке",  # Дательный падеж
            "скидку",  # Винительный падеж
            "скидкой",  # Творительный падеж
            'скидок',
            'скидочка',
            )

rub_nums = ("сто тысяч",
            "двести тысяч",
            "триста тысяч",
            "пятьсот тысяч",
            "шестьсот тысяч",
            "миллион",
            "полтора миллиона",
            )

rub_text = ('рублей', 'рубли', 'руб', 'рубля', '')

num_texts = 5500

# Генерация нескольких текстов с различными скидками
generated_texts = []
for _ in tqdm(range(num_texts), total=num_texts):  # Генерация текстов
    prompt = random.choice(prompt_templates)
    generated_texts.extend(generate_text(prompt, max_length=50 + random.randint(0, 400),
                                         num_return_sequences=1))

fake_words = generate_fake_words(num_texts=num_texts)

df = pd.DataFrame(columns=['processed_text', 'target_labels_positions'])

for idx_rec, (text, fake) in enumerate(zip(generated_texts, fake_words)):

    words = re.findall('[а-яё]+', text.lower())

    labels = defaultdict(list)

    # В каждую третью запись будем лепить скидки
    if not idx_rec % 3 or not idx_rec % 5 or not idx_rec % 7:

        if len(fake) < 10:
            words = fake + words
        else:
            idx = random.randint(5, len(fake) - 5) // 2
            words = fake[:idx] + words + fake[idx:]

        count_discnt = random.choice([1, 2])
        for nub_cnt in range(count_discnt):
            number = random.choice(nums + rub_nums)
            if number in nums:
                prs = random.choice(percents)
            else:
                prs = random.choice(rub_text)

            number = f'{number} {prs}'.strip().split()  # процент скидки
            discnt = random.choice(discount)  # слово скидка
            place = random.choice([0, 1])  # расположения слова "скидка": 1 -> после процента
            diffs = random.choice([1, 2, 1, 3, 2, 1])  # смещение слова "скидка" от процентов

            # Первая итерация: начальная и конечная позиция скидки в тексте
            fst_pos = 3
            if not nub_cnt and count_discnt > 1:
                last_pos = (len(words) - 5) // 2
            else:
                if count_discnt > 1:
                    fst_pos = last_pos + 8
                last_pos = len(words) - len(number) - diffs - 2

            start_idx = random.randint(fst_pos, last_pos)
            disc_idx = start_idx + len(number) + diffs if place else start_idx - diffs

            labels['B-discount'].append(disc_idx)
            words[disc_idx] = discnt

            for i, txt_num in enumerate(number):
                if not i:
                    labels['B-value'].append(start_idx)
                else:
                    labels['I-value'].append(start_idx + i)
                words[start_idx + i] = txt_num

    text = ' '.join(words)

    df.loc[len(df)] = [text, dict(labels)]

df.iloc[:5000].to_csv('fake_train_add.csv', index=False)
df.iloc[5000:].to_csv('fake_test_add.csv', index=False)

  2%|▏         | 79/3333 [08:01<6:29:29,  7.18s/it]

In [ ]:
from IPython.display import display, FileLink
from zipfile import ZipFile, ZIP_DEFLATED as ZD
from glob import glob

files = glob(f'*.csv')
zip_filename = 'fake_text.zip'
with ZipFile(zip_filename, 'w',  compression=ZD, compresslevel=9) as zip_file:
    for filename in files:
        print(filename)
        zip_file.write(filename)
FileLink(zip_filename)